Copyright 2021, Badhan Kumar Das, All rights reserved

In [ ]:
import pixellib
from pixellib.torchbackend.instance import instanceSegmentation
import cv2
import numpy as np
from collections import Counter

In [ ]:
model = instanceSegmentation()
model.load_model("pointrend_resnet50.pkl")
capture = cv2.VideoCapture("thairoad2.mp4")
width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
codec = cv2.VideoWriter_fourcc(*'DIVX')
save_video = cv2.VideoWriter("thairoad2output_3.mp4", codec, 3, (1280, 720))

In [ ]:
c = Counter(car=0, truck=0, bus=0, motorcycle=0)
x_pre = -10

while True:
    ret, frame = capture.read()
    if ret:
        res,output = model.segmentFrame(frame, show_bboxes=True)
        boxes_ids = list(res.values())[0]
        classid = list(res.values())[2]
        i = 0
        for box_id in boxes_ids:
            x,y,w,h = box_id
    #########   modify the range according to tracking line ########
            if x not in range(x_pre-50,x_pre+50) and y in range(390,405):
                c[classid[i]]+=1
                x_pre = x
            i=i+1
        
        carcount = c['car']
        truckcount = c['truck']
        buscount = c['bus']
        motorcyclecount = c['motorcycle']

        cv2.putText(output,f'Car passed:{carcount}',(25,640),cv2.FONT_HERSHEY_SIMPLEX,0.6,(200,0,0),2)
        cv2.putText(output,f'Truck passed:{truckcount}',(25,660),cv2.FONT_HERSHEY_SIMPLEX,0.6,(200,0,0),2)
        cv2.putText(output,f'Bus passed:{buscount}',(25,680),cv2.FONT_HERSHEY_SIMPLEX,0.6,(200,0,0),2)
        cv2.putText(output,f'Motorcycle passed:{motorcyclecount}',(25,700),cv2.FONT_HERSHEY_SIMPLEX,0.6,(200,0,0),2)
        
    ########### tracking line #########
    cv2.line(output,(50,395),(1200,395),(0,0,100),2)
    cv2.imshow("Output",output)
    save_video.write(output)
    
    key = cv2.waitKey(30)
    if key==27:
        break

save_video.release()
capture.release()
cv2.destroyAllWindows()
        